## Normal shading with textures

This example shows how to:
   - modify predefined materials
   - use displacement map
   - apply large textures to material
   
Tkinter GUI window is launched from the notebook in this example. This allows re-running code cells and see results without scrolling back to an inlined figure.

![notebook output image](https://plotoptix.rnd.team/images/normal_shading_with_textures.jpg "This notebook output")

In [1]:
import numpy as np
from plotoptix import TkOptiX
from plotoptix.materials import m_clear_glass, m_plastic # predefined materials
from plotoptix.utils import map_to_colors, make_color_2d, read_image, simplex

Make some data first.

In [2]:
m = 300
r = 0.08 * np.random.rand(m) + 0.01
p = 3 * (np.random.rand(m,3) - 0.5)
p[:,0] *= 0.8
p[:,2] *= 1.5
p[:,1] = r[:]

Setup the raytracer using Tkinter GUI as the output target.

In [3]:
rt = TkOptiX()
rt.set_param(min_accumulation_step=4,    # set more accumulation frames to get rid of the noise
             max_accumulation_frames=512,
             light_shading="Hard")       # use ligth shading best for caustics
rt.set_uint("path_seg_range", 5, 10)     # more path segments to allow multiple reflections

Only *diffuse* material is available by default. Other materials need to be configured before using.

In [4]:
import copy
m_beads = copy.deepcopy(m_plastic) # a new material based on predefined properties

rt.setup_material("plastic", m_plastic)
rt.setup_material("glass", m_clear_glass)
rt.setup_material("beads", m_beads)

Add objects to the scene.

**Note 1:** particles with ``geom="ParticleSetTextured"`` geometry can have 3D orientation, provided with ``u`` and ``v`` arguments, required for applying textures. 3D oreintation is randomized if these vectors are omitted.

**Note 2:** ``geom_attr="DisplacedSurface"`` is used to allow for displacement mapping.

In [5]:
rt.set_data("plane", geom="Parallelograms", mat="plastic",
            pos=[-3, 0, -3], u=[6, 0, 0], v=[0, 0, 6], c=0.8)

rt.set_data("sphere", geom="ParticleSetTextured", mat="glass", geom_attr="DisplacedSurface",
            pos=[0.0, 0.7, 0.0], u=[-1, 0, 0], r=0.4, c=10)

rt.set_data("particles", geom="ParticleSetTextured", mat="beads",
            pos=p, r=r, c=0.95)

Setup a good point of view, set background and lights.

In [6]:
rt.setup_camera("cam1", cam_type="DoF",
                eye=[-2.1, 2.4, 0], target=[0, 0, 0], up=[0.28, 0.96, 0.05],
                aperture_radius=0.01, fov=30, focal_scale=0.91)

rt.setup_light("light1", pos=[4, 5.1, 3], color=[12, 11, 10], radius=1.9)
rt.setup_light("light2", pos=[-1.5, 3, -2], color=[8, 9, 10], radius=0.2)
rt.set_background(0)
rt.set_ambient(0)

exposure = 0.4; gamma = 2.2 
rt.set_float("tonemap_exposure", exposure)
rt.set_float("tonemap_gamma", gamma)
rt.set_float("denoiser_blend", 0.25)
rt.add_postproc("Denoiser")  # apply gamma correction postprocessing stage, or
#rt.add_postproc("Gamma")    # use AI denoiser (exposure and gamma are applied as well)

Open the GUI.

In [7]:
rt.start()

Make a gradient texture, use it with the *glass* material:

In [8]:
y = np.linspace(0, 1, 100)
M = np.stack((y,y)).T

In [9]:
M1 = map_to_colors(-M, "Purples")
M1 = make_color_2d(M1, gamma=gamma, channel_order="RGBA")
rt.set_texture_2d("purple", 10*M1)

m_clear_glass["ColorTextures"] = [ "purple" ]
m_clear_glass["VarFloat3"]["refraction_index"] = [1.4, 1.4, 1.4]
rt.update_material("glass", m_clear_glass, refresh=True)

Make a gradient with another color map for the plastic particles.

In [10]:
M2 = map_to_colors(M, "RdYlBu")
M2 = make_color_2d(M2, gamma=gamma, channel_order="RGBA")
rt.set_texture_2d("redyellowblue", M2)

m_beads["ColorTextures"] = [ "redyellowblue" ]
rt.update_material("beads", m_beads, refresh=True)

Calculate displacement map over 2D plane using simplex noise:

In [11]:
nn = 2000
x = np.linspace(0, 50, nn)
z = np.linspace(0, 50, nn)

X, Z = np.meshgrid(x, z)
XZ = np.stack((X.flatten(), Z.flatten(), np.full(nn**2, 1.0, dtype=np.float32))).T.reshape(nn, nn, 3)
XZ = np.ascontiguousarray(XZ, dtype=np.float32)
Y = simplex(XZ)
Y = np.sin(10 * Y)

Use the displacement map for the shading normal tilt. Displacement is relative to the object size, it should be a small value to look like a wrinkles on the plane. Shading modulation is implemented as a ``Float2`` texture calculated from the displacement, and accompanied with an inverse aspect ratio, ``normaltilt_iar``, parameter. Use ``set_normal_tilt()`` method to set these two parameters.

In [12]:
rt.set_normal_tilt("plastic", 0.00008*Y, refresh=True)

OK, to make it more interesting, let's add some lines along wrinkles. An RGBA texture is needed for this.

In [13]:
Ym = 0.5 * (np.copy(Y) + 1)
m = (Ym > 0.45) & (Ym < 0.55)
Ym[m] = 0.0
Ym[~m] = 0.95

In [14]:
M3 = make_color_2d(Ym, channel_order="RGBA")
rt.set_texture_2d("ripples", M3)

m_plastic["ColorTextures"] = [ "ripples" ]
rt.update_material("plastic", m_plastic, refresh=True)

Surface of spheres can be actually displaced, which gives accurate look of a strongly modified object, not only an impression of inclined surface.

In [15]:
#D = read_image("data/blur.png", normalized=True).mean(axis=2)
#rt.set_displacement("sphere", 0.7 + 0.3*D, refresh=True)

rt.load_displacement("sphere", "data/blur.png", prescale=0.3, baseline=0.7, refresh=True)

Close GUI window, release resources.

In [16]:
rt.close()